# Get BERT vectors from a BERT model

In [ ]:
%tensorflow_version 1.x

In [ ]:
!git clone https://github.com/gaphex/bert_experimental/

import tensorflow as tf
import numpy as np
import sys
import pandas as pd
import random



sys.path.insert(0, "/content/bert_experimental")

from bert_experimental.finetuning.text_preprocessing import build_preprocessor
from bert_experimental.finetuning.graph_ops import load_graph

In [ ]:
print(tf.__version__)

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

In [ ]:
#auth google
#from google.colab import auth
#auth.authenticate_user()

In [ ]:
#!gsutil cp -R gs://mann_bucket/model/frozen_graph_121_mod2.pb .


In [ ]:
#!gsutil cp -R gs://mann_bucket/data1.csv .
from google.colab import files
uploaded = files.upload()


In [ ]:
df = pd.read_csv("dictionary.csv",sep=";",header = 0,encoding = "UTF-8")
colName = 'title' #@param {type:"string"}
#df = df[[colName]]
print('df -- ',df.shape)
df.head()

In [ ]:

restored_graph = load_graph("frozen_graph_121_mod2.pb")

graph_ops = restored_graph.get_operations()
input_op, output_op = graph_ops[1].name, graph_ops[3068].name
print(input_op, output_op)
x = restored_graph.get_tensor_by_name(input_op + ':0')
y = restored_graph.get_tensor_by_name(output_op + ':0')
preprocessor = build_preprocessor("./uncased_L-12_H-768_A-12/vocab.txt", 64)

In [ ]:
print(graph_ops)

In [ ]:
def index_marks(nrows, chunk_size):
    return range(1 * chunk_size, (nrows // chunk_size + 1) * chunk_size, chunk_size)

def split(dfm, chunk_size):
    indices = index_marks(dfm.shape[0], chunk_size)
    return np.split(dfm, indices)

In [ ]:
def bert_encode(d):
    
    print("Shape: {}; {}".format(d.shape, d.index))
    sentences = []
    for strObj in d['title']:
        sentences.append(strObj)
#    Bert as a service
    py_func = tf.numpy_function(preprocessor, [x], [tf.int32, tf.int32, tf.int32], name='preprocessor')
    py_func = tf.numpy_function(preprocessor, [x], [tf.int32, tf.int32, tf.int32])
    with tf.Session(graph=restored_graph) as sess :
        py_func = tf.numpy_function(preprocessor, [x], [tf.int32, tf.int32, tf.int32], name='preprocessor')
        py_func = tf.numpy_function(preprocessor, [x], [tf.int32, tf.int32, tf.int32])    
        vectors = sess.run(y, feed_dict={
                x: np.array(sentences).reshape(-1,1)
            })
    if (len(sentences) != len(vectors)):
        print(sentences)
    
    
    cols = [None] * 50
    cols[0:50] = ('X' + str(s) for s in range(1,51))
    res = pd.DataFrame(vectors, columns = cols)
    return res

In [ ]:
import datetime
max_seq_len = min(1000, len(df.index))

print('df -- ',df.shape)

indices = list(index_marks(df.shape[0], max_seq_len))
print("Marks: {}".format(indices))

chunks = split(df, max_seq_len)
   
for i in range(0, len(chunks)):
    startTime = datetime.datetime.now()
    print(i)
    if i == 0 :
        res = bert_encode(chunks[i])
    else :
        res = res.append(bert_encode(chunks[i]))
    print("total time taken: ", str(datetime.datetime.now()-startTime))
dff = pd.concat([df.reset_index(drop=True), res.reset_index(drop=True)], axis=1)
  
  
#dff.drop_duplicates(keep=False,inplace=True)
print('result -- ',dff.shape)

In [ ]:
dff['code']=df['code']
dff = dff.filter(regex='^((?!title).)*$')

In [ ]:
dff.to_csv("result_bert_encode_121_mod2.csv.gz",sep=";",encoding="UTF-8",index = False, compression = 'gzip') 
#from google.colab import files
#files.download('result_bert_encode.csv.gz')
#!gsutil cp -r 'mann_frozen_graph.pb' gs://mann_bucket/model
#!gsutil cp -r 'result_bert_encode_121_mod2.csv.gz' gs://mann_bucket